In [9]:
script_path = getwd()                                # %exclude_jupyterlab%

In [10]:
script_path                                          # %exclude_jupyterlab%

[1] "/Volumes/ccrsf-ifx/Software/github/SF_scMaestro/workflow/scripts/rna"

In [1]:
system(paste0('Rscript ', script_path,                # %exclude_jupyterlab%
        '/sc_singleR_opt.R --genome="hg38" --markerList="/Volumes/ccrsf-ifx/Software/scripts/bin/currentsnake/single_cell/gene_lists/human_gene_list.csv" --outdir="/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/singleR" --rds="/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/test_dir/seur_10x_cluster_object.rds"'),  # %exclude_jupyterlab%
       intern = T) # %exclude_jupyterlab%

ERROR: Error in paste0("Rscript ", script_path, "/sc_singleR_opt.R --genome=\"hg38\" --markerList=\"/Volumes/ccrsf-ifx/Software/scripts/bin/currentsnake/single_cell/gene_lists/human_gene_list.csv\" --outdir=\"/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/singleR\" --rds=\"/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/test_dir/seur_10x_cluster_object.rds\""): object 'script_path' not found


In [9]:
opt = readRDS("/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/singleR/opt.rds")         # %exclude_jupyterlab%

In [10]:
opt

$rds
[1] "/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/test_dir/seur_10x_cluster_object.rds"

$sampleid
[1] "scRNA"

$genome
[1] "hg38"

$markerList
[1] "/Volumes/ccrsf-ifx/Software/scripts/bin/currentsnake/single_cell/gene_lists/human_gene_list.csv"

$outdir
[1] "/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/singleR"

$help
[1] FALSE

In [11]:
system(paste0("mkdir -p ", opt$outdir))
setwd(opt$outdir) 

In [12]:
library(Seurat)
library(SingleR)
library(celldex)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    ro

In [13]:
seur <- readRDS(opt$rds)

In [14]:
sce <- as.SingleCellExperiment(seur)


Warning message:
“Layer ‘data’ is empty”
Warning message:
“Layer ‘scale.data’ is empty”


In [15]:
sce <- scater::logNormCounts(sce)

In [16]:
geneList <- read.csv(opt$markerList, header=FALSE, row.names=1, stringsAsFactors = FALSE)

In [23]:
## plotScoreDistribution: use this plot to check that outlier detection in pruneScores() behaved sensibly. 
## Labels with especially low deltas may warrant some additional caution in their interpretation.
generatePlots <- function(name, pred, seur, markers){
  saveRDS(pred, paste0('pred_', name, '.rds'))
  #pdf(paste0('heatmap_', name, '.pdf'))
  #print(plotScoreHeatmap(pred, clusters=sce$res.0.1))
  #dev.off()
  pdf(paste0('ScoreDistribution_', name, '.pdf'), height=15)
  # plotScoreDistribution deprecated
  #print(plotScoreDistribution(pred, show = "delta.med", ncol = 3, show.nmads = 3))
  print(plotDeltaDistribution(pred, show = "delta.med", ncol = 3))
  dev.off()
  seur@meta.data[name] <- ''
  seur@meta.data[rownames(pred),name] <- pred$pruned.labels
  Idents(seur) <- seur@meta.data[,name]
  pdf(paste0("UMAP_", name, ".pdf"))
  print(DimPlot(seur))
  dev.off()
  pdf(paste0("TSNE_", name, ".pdf"))
  print(DimPlot(seur, reduction='tsne'))
  dev.off()
  dir.create("gene_list_plots")
  for (i in 1:dim(markers)[1]) {
    genes.use <- as.character(markers[row.names(markers)[i],])
    numGenes <- length(genes.use[genes.use %in% row.names(seur)])
    if (sum(genes.use %in% row.names(seur)) > 1) {
      png(paste0('gene_list_plots/ViolinPlot_', name, '_', row.names(geneList)[i],'.png'), height = 5, width = (length(genes.use[genes.use %in% row.names(seur)])+1)*5, units='in', res=300)
      print(suppressWarnings(VlnPlot(seur, genes.use[genes.use %in% row.names(seur)], ncol=numGenes)))
      dev.off()
    }
  }
  return(seur)
}

In [26]:
generateMarkerPlots <- function(seur, markers) {
  dir.create("gene_list_plots")
  for (i in 1:dim(markers)[1]) {
    genes.use <- as.character(markers[row.names(markers)[i],])
    numGenes <- length(genes.use[genes.use %in% row.names(seur)])
    if (sum(genes.use %in% row.names(seur)) > 1) {
      png(paste0('gene_list_plots/FeaturePlot_', row.names(geneList)[i],'.png'), height = 5, width = (length(genes.use[genes.use %in% row.names(seur)])+1)*5, units='in', res=300)
      print(suppressWarnings(FeaturePlot(seur, genes.use[genes.use %in% row.names(seur)], ncol=numGenes)))
      dev.off()
    }
  }
}

In [27]:
# The objects returned by functions:  ImmGenData, MouseRNAseqData, 
# HumanPrimaryCellAtlasData and BlueprintEncodeData are saved under 
# /home/docker/. So you only need to load the RDS files directly. 
if (opt$genome == "mm10") {
  #immgen <- ImmGenData()# - mouse
  #mouserna <- MouseRNAseqData()# - mouse
  immgen <- readRDS("/home/docker/ImmGenData.rds")
  mouserna <- readRDS("/home/docker/MouseRNAseqData.rds")
  pred.multi <- SingleR(test = sce,
      ref = list(IG=immgen, MRNA=mouserna),
      labels = list(immgen$label.main, mouserna$label.main))
  seur <- generatePlots('immgen_mourserna', pred.multi, seur, geneList)
  temp <- table(pred.multi$pruned.labels)
  write.csv(sort(temp, decreasing=TRUE), 'annotations_Immgen-MouseRNA_general.csv', row.names=FALSE)

  pred.immgen <- SingleR(test = sce,
      ref = immgen, labels = immgen$label.main)
  seur <- generatePlots('immgen', pred.immgen, seur, geneList)
  temp <- table(pred.immgen$pruned.labels)
  write.csv(sort(temp, decreasing=TRUE), 'annotations_Immgen_general.csv', row.names=FALSE)

  pred.mouserna <- SingleR(test = sce,
      ref = mouserna, labels = mouserna$label.main)
  seur <- generatePlots('mourserna', pred.mouserna, seur, geneList)
  temp <- table(pred.mouserna$pruned.labels)
  write.csv(sort(temp, decreasing=TRUE), 'annotations_MouseRNA_general.csv', row.names=FALSE)

  write.csv(seur@meta.data[,c('immgen_mourserna', 'immgen', 'mourserna')], 'annotations_compiled_barcode.csv', quote=FALSE)
} else if (opt$genome == "hg38") {
  #hpca <- HumanPrimaryCellAtlasData()
  #blueprint <- BlueprintEncodeData()
  hpca <- readRDS("/home/docker/HumanPrimaryCellAtlasData.rds")
  blueprint <- readRDS("/home/docker/BlueprintEncodeData.rds")
  pred.multi <- SingleR(test = sce,
      ref = list(BP=blueprint, HPCA=hpca),
      labels = list(blueprint$label.main, hpca$label.main))
  seur <- generatePlots('blueprintencode_hpca', pred.multi, seur, geneList)
  temp <- table(pred.multi$pruned.labels)
  write.csv(sort(temp, decreasing=TRUE), 'annotations_BlueprintENCODE-HPCA_general.csv', row.names=FALSE)

  pred.blueprint <- SingleR(test = sce,
      ref = blueprint, labels = blueprint$label.main)
  seur <- generatePlots('blueprintencode', pred.blueprint, seur, geneList)
  temp <- table(pred.blueprint$pruned.labels)
  write.csv(sort(temp, decreasing=TRUE), 'annotations_BlueprintENCODE_general.csv', row.names=FALSE)

  pred.hpca <- SingleR(test = sce,
      ref = hpca, labels = hpca$label.main)
  seur <- generatePlots('hpca', pred.hpca, seur, geneList)
  temp <- table(pred.hpca$pruned.labels)
  write.csv(sort(temp, decreasing=TRUE), 'annotations_HPCA_general.csv', row.names=FALSE)

  write.csv(seur@meta.data[,c('blueprintencode_hpca', 'blueprintencode', 'hpca')], 'annotations_compiled_barcode.csv', quote=FALSE)
}

suppressWarnings(generateMarkerPlots(seur, geneList))
saveRDS(seur, 'seur_10x_cluster_singler.rds')


Warning message:
“Groups with fewer than two datapoints have been dropped.
ℹ Set `drop = FALSE` to consider such groups for position adjustment purposes.”
Warning message in max(data$density, na.rm = TRUE):
“no non-missing arguments to max; returning -Inf”
Warning message:
“Computation failed in `stat_ydensity()`.
Caused by error in `$<-.data.frame`:
! replacement has 1 row, data has 0”
Warning message:
“Groups with fewer than two datapoints have been dropped.
ℹ Set `drop = FALSE` to consider such groups for position adjustment purposes.”
Warning message in max(data$density, na.rm = TRUE):
“no non-missing arguments to max; returning -Inf”
Warning message:
“Computation failed in `stat_ydensity()`.
Caused by error in `$<-.data.frame`:
! replacement has 1 row, data has 0”


TableGrob (2 x 1) "arrange": 2 grobs
  z     cells    name           grob
1 1 (1-1,1-1) arrange gtable[layout]
2 2 (2-2,1-1) arrange gtable[layout]


Warning message in dir.create("gene_list_plots"):
“'gene_list_plots' already exists”
Warning message:
“Groups with fewer than two datapoints have been dropped.
ℹ Set `drop = FALSE` to consider such groups for position adjustment purposes.”
Warning message:
“Groups with fewer than two datapoints have been dropped.
ℹ Set `drop = FALSE` to consider such groups for position adjustment purposes.”
Warning message:
“Groups with fewer than two datapoints have been dropped.
ℹ Set `drop = FALSE` to consider such groups for position adjustment purposes.”
Warning message:
“Groups with fewer than two datapoints have been dropped.
ℹ Set `drop = FALSE` to consider such groups for position adjustment purposes.”
Warning message:
“Groups with fewer than two datapoints have been dropped.
ℹ Set `drop = FALSE` to consider such groups for position adjustment purposes.”
Warning message:
“Groups with fewer than two datapoints have been dropped.
ℹ Set `drop = FALSE` to consider such groups for position adjustm

In [20]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 22.04.4 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3 
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/libopenblasp-r0.3.20.so;  LAPACK version 3.10.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] celldex_1.12.0              SingleR_2.4.1              
 [3] SummarizedExperiment_1.32.0 Biobase_2.62.0             
 [5] GenomicRanges_1.54.1        G

In [11]:
notebook_prefix = "sc_singleR"                                                         # %exclude_jupyterlab%
notebook_name = paste0(notebook_prefix, ".ipynb")                                     # %exclude_jupyterlab%
notebook_r = paste0(script_path, "/", paste0(notebook_prefix, ".r"))                  # %exclude_jupyterlab%
notebook_path = paste0(script_path, "/", notebook_name)                               # %exclude_jupyterlab%
opt_name = paste0(script_path, "/", sub(".ipynb", "_opt.R", notebook_name))           # %exclude_jupyterlab%
output = paste0(script_path, "/", sub(".ipynb", ".prod.R", notebook_name))            # %exclude_jupyterlab%
cmd1 = paste0("jupyter nbconvert --to script --output ",                              # %exclude_jupyterlab%
             notebook_prefix, ' ', notebook_path, "> /dev/null 2>&1 ")                # %exclude_jupyterlab%
cmd1                                                                                  # %exclude_jupyterlab%
system(cmd1, intern = TRUE)                                                            # %exclude_jupyterlab%

[1] "jupyter nbconvert --to script --output sc_singleR /Volumes/ccrsf-ifx/Software/github/SF_scMaestro/workflow/scripts/rna/sc_singleR.ipynb> /dev/null 2>&1 "

character(0)

In [12]:
cmd2 = paste0('cat -s ', opt_name, ' ', notebook_r,                                      # %exclude_jupyterlab%
             ' |grep -v exclude_jupyterlab > ', output,  ' 2>&1')                     # %exclude_jupyterlab%
cmd2                                                                                  # %exclude_jupyterlab%
system(cmd2, intern = T)                                                              # %exclude_jupyterlab%
system(paste0("rm ", notebook_r))                                                     # %exclude_jupyterlab%  

[1] "cat -s /Volumes/ccrsf-ifx/Software/github/SF_scMaestro/workflow/scripts/rna/sc_singleR_opt.R /Volumes/ccrsf-ifx/Software/github/SF_scMaestro/workflow/scripts/rna/sc_singleR.r |grep -v exclude_jupyterlab > /Volumes/ccrsf-ifx/Software/github/SF_scMaestro/workflow/scripts/rna/sc_singleR.prod.R 2>&1"

character(0)